# Bluesky Device for APS taxi & fly scans

Some EPICS fly scans at APS are triggered by a pair of EPICS
[*busy*](https://epics-modules.github.io/busy/) records.  The first *busy*
record is called `taxi` and is responsible for preparing the hardware to fly.
Once *taxi* is complete, the second *busy* record called `fly` performs the actual fly scan.
In a third (optional) phase, data is collected from hardware and written to a
file.

Compare the taxi & fly scan algorithm to an airplane flight:

phase | airplane flight | taxi & fly scan
--- | --- | ---
preparation | ticketing & boarding | configuration of software
taxi | move the aircraft to the end of the runway | move the hardware to pre-scan positions
fly | start moving, liftoff at flight velocity | start moving, begin collecting data at first position

## Bluesky (Python)setup

Here are the packages to import.  The first block contains Python standard
packages, then come the various bluesky packages.  Just the parts we plan on
using here.

* Create a logger instance in case we want to investigate internal details as our code runs.
* Create an instance of the bluesky `RunEngine`.
* Create a temporary databroker catalog to save collected data.
* Subscribe the catalog to receive all data published by the RunEngine.

In [1]:
import logging

from apstools.synApps import SseqRecord
from apstools.plans import run_blocking_function
import bluesky
import bluesky.plan_stubs as bps
import databroker
from ophyd import Component, Device, EpicsSignal

logger = logging.getLogger()
logging.basicConfig(
    level=logging.INFO,  # more details than default (WARNING) level
    format="%(asctime)s %(levelname)s %(name)s - %(message)s"
)

RE = bluesky.RunEngine()
cat = databroker.temp().v2
RE.subscribe(cat.v1.insert)
# RE.msg_hook = bluesky.utils.ts_msg_hook  # deeper logging details

/home/prjemian/.conda/envs/bluesky_2023_3/lib/python3.11/site-packages/apstools/devices/aps_data_management.py


0

## EPICS IOC

We'll start with an EPICS IOC that provides two instances of the
[*busy*](https://epics-modules.github.io/busy/) record and two instances of the
[*sseq*](https://htmlpreview.github.io/?https://raw.githubusercontent.com/epics-modules/calc/R3-6-1/documentation/sseqRecord.html)
record.

In the `gp:` IOC, we can use these general purpose PVs for this example:

PV | record | purpose
--- | --- | ---
`gp:mybusy1` | *busy* | taxi (preparation) phase
`gp:mybusy2` | *fly* | fly (fly scan) phase
`gp:userStringSeq1` | *sseq* | actions for the taxi phase
`gp:userStringSeq2` | *sseq* | actions for the fly phase

The EPICS _busy_ record is used to start a phase (taxi or fly) and report when that
phase is finished. The EPICS _sseq_ record describes and manages the work to be done
in that phase. In this example, the work is simple (wait for some seconds). In
other situations, the work could involve a complex setup of software to collect data
or a complete
[_sscan_](https://epics.anl.gov/bcda/synApps/sscan/sscanRecord.html) process.

We need to configure these records for our use. We'll use the ophyd support for
[sseq](https://bcda-aps.github.io/apstools/latest/api/synApps/_sseq.html)
records from [apstools](https://bcda-aps.github.io/apstools). For the _busy_
records, we choose a simpler connections than the
[busy](https://bcda-aps.github.io/apstools/latest/api/synApps/_busy.html)
support from apstools. Instead, we'll connect with the `.OUT` and `.VAL` fields
using `EpicsSignal` (from ophyd). This allows us to add the `put_complete=True`
keyword argument that we need for the `.VAL` field's connection. The
`put_complete` keyword provides prompt notification when the asynchronous _busy_
record is finished.

We'll connect ophyd structures with these PVs and make bluesky plans here to
setup the PVs for this demo and to run the taxi+fly scan.

**NOTE**: This example does not collect any data. It only shows how the taxi &
fly scan is operated from bluesky.

### EPICS: How do the *busy* and *sseq* records interact in this demo?

Summary: The *busy* record tells the *sseq* record to do all its processing
steps.  The *sseq* record waits its assigned time, then turns the *busy* record
off.

Details:
The user changes the *busy* record's `.VAL` field from 0 to 1.  This causes the
*busy* record to send a 1 to the PV named in its `.OUT` field.  That field
contains the *sseq* record's `.PROC` field.  Thus, it causes the *sseq* record
to begin its processing steps.

There are two steps for the *sseq* record:

1. Wait for the assigned time.
2. Tell the *busy* record it is done.

One additional action was needed, to remove the PV name from the *busy* record's
`.OUT` field.  Without this additional action, the *sseq* record will be
processed by every change to the *busy* record value.

All of this configuration is programmed in the `setup_plan()` in the
`TaxiFlyScanDevice` class below.

### Call a blocking function from a bluesky plan

The `SseqRecord()` support in apstools has a `reset()` method to clear any
previous settings of the EPICS PVs and ophyd object and return them to default
settings.  This method is written as ophyd code, intended to be called from a
command-line session.  The commands it contains that may take some time to
complete and possibly block the normal execution of the RunEngine's callback
thread. The
[`run_blocking_function()`](https://bcda-aps.github.io/apstools/latest/api/_plans.html#module-apstools.plans.run_blocking_function_plan)
plan from *apstools* allows us to run `reset()` in a thread so that it does not
block the `RunEngine`.


## Define the TaxiFlyScanDevice

All of the taxi and fly components are configuration details and should not be included with any data to be recorded.  They are all marked with the `kind="config"` keyword.

In [2]:
class TaxiFlyScanDevice(Device):
    taxi = Component(EpicsSignal, "mybusy1", put_complete=True, kind="config")
    taxi_out = Component(EpicsSignal, "mybusy1.OUT", kind="config")
    taxi_sseq = Component(SseqRecord, "userStringSeq1", kind="config")

    fly = Component(EpicsSignal, "mybusy2", put_complete=True, kind="config")
    fly_out = Component(EpicsSignal, "mybusy2.OUT", kind="config")
    fly_sseq = Component(SseqRecord, "userStringSeq2", kind="config")

    def setup_plan(self, t_taxi, t_fly):
        logger.info("taxi time: %.2f s", t_taxi)
        logger.info("fly time: %.2f s", t_fly)
        yield from bps.mv(
            self.taxi, 0,
            self.fly, 0,
        )
        yield from bps.mv(
            self.taxi_out, f"{self.taxi_sseq.prefix}.PROC CA NMS",
            self.fly_out, f"{self.fly_sseq.prefix}.PROC CA NMS",
        )

        # Clear any previous setup of the sseq records.
        # The reset function is not a plan.
        yield from run_blocking_function(self.taxi_sseq.reset)
        yield from run_blocking_function(self.fly_sseq.reset)

        yield from bps.mv(
            # Set the delay time.
            self.taxi_sseq.steps.step1.delay, t_taxi,
            # Remove the .PROC poke so the sseq will not be triggered again.
            self.taxi_sseq.steps.step1.string_value, "",
            self.taxi_sseq.steps.step1.output_pv, f"{self.taxi_out.pvname} CA NMS",
            # Mark the busy record as Done.
            self.taxi_sseq.steps.step2.string_value, self.taxi.enum_strs[0],
            self.taxi_sseq.steps.step2.output_pv, f"{self.taxi.pvname} CA NMS",

            # Similar setup for the fly.
            self.fly_sseq.steps.step1.delay, t_fly,
            self.fly_sseq.steps.step1.string_value, "",
            self.fly_sseq.steps.step1.output_pv, f"{self.fly_out.pvname} CA NMS",
            self.fly_sseq.steps.step2.string_value, self.fly.enum_strs[0],
            self.fly_sseq.steps.step2.output_pv, f"{self.fly.pvname} CA NMS",
        )

    def taxi_fly_plan(self, time_taxi=5, time_fly=5):
        logger.info("before setup_plan")
        yield from self.setup_plan(time_taxi, time_fly)

        logger.info("before taxi")
        yield from bluesky.plan_stubs.mv(self.taxi, self.taxi.enum_strs[1])
        logger.info("after taxi")

        logger.info("before fly")
        yield from bluesky.plan_stubs.mv(self.fly, self.fly.enum_strs[1])
        logger.info("after fly")

## Create a Python object.

Here, the PVs are from the `gp:` IOC.

In [3]:
ifly = TaxiFlyScanDevice("gp:", name="ifly")
ifly.wait_for_connection()

## Run the complete plan

Run the `taxi_fly_plan()` with the bluesky RunEngine. There is no data to be
collected in this example, only logging entries.  Wait 2 seconds in the taxi
phase, then 3 seconds in the fly phase.

In [4]:
RE(ifly.taxi_fly_plan(2, 3))

2023-10-23 17:03:39,025 INFO bluesky - Executing plan <generator object TaxiFlyScanDevice.taxi_fly_plan at 0x7f3e7d11e7a0>
2023-10-23 17:03:39,127 INFO bluesky.RE.state - Change state on <bluesky.run_engine.RunEngine object at 0x7f3e81f38a10> from 'idle' -> 'running'
2023-10-23 17:03:39,128 INFO root - before setup_plan
2023-10-23 17:03:39,128 INFO root - taxi time: 2.00 s
2023-10-23 17:03:39,129 INFO root - fly time: 3.00 s
2023-10-23 17:03:39,191 INFO root - before taxi
2023-10-23 17:03:41,190 INFO root - after taxi
2023-10-23 17:03:41,192 INFO root - before fly
2023-10-23 17:03:44,190 INFO root - after fly
2023-10-23 17:03:44,191 INFO bluesky.RE.state - Change state on <bluesky.run_engine.RunEngine object at 0x7f3e81f38a10> from 'running' -> 'idle'
2023-10-23 17:03:44,191 INFO bluesky - Cleaned up from plan <generator object TaxiFlyScanDevice.taxi_fly_plan at 0x7f3e7d11e7a0>


()